In [1]:
# --- Celda 1: Importación de Librerías ---
import requests
import pandas as pd
import time
import os
import json
import logging
from dotenv import load_dotenv
from collections import deque
import googlemaps # <--- LIBRERÍA NUEVA

logging.info("Librerías importadas.")



In [7]:
# --- Celda 2: Configuraciones Básicas ---
# Asegura que no se dupliquen los handlers si se corre la celda múltiples veces
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("001_cali_api.log"),
        logging.StreamHandler()
    ]
)

logging.info("Inicio del notebook 001_cali_api.ipynb.")

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
logging.info("Configuraciones de Pandas para visualización aplicadas.")

2025-10-07 00:24:37,871 - INFO - Inicio del notebook 001_cali_api.ipynb.
2025-10-07 00:24:37,872 - INFO - Configuraciones de Pandas para visualización aplicadas.


In [8]:
# --- Celda 3: Carga de Variables de Entorno y Rutas (Modificada) ---

# Ruta al archivo .env (ajústala si es necesario)
ENV_FILE_PATH = '../.env/.env' 

# RUTA DE SALIDA: donde se guardará el nuevo CSV con los datos de la API
RAW_API_CSV_PATH = '../datos_brutos/api/001_Local_cali_api.csv'

# Ruta para el archivo JSON que guardará el estado de la extracción
STATE_FILE_PATH = '../datos_brutos/api/001_Local_cali_api.json'

# --- El resto del código de esta celda para cargar las claves API se mantiene igual ---
logging.info(f"Cargando variables de entorno desde: {ENV_FILE_PATH}")
if os.path.exists(ENV_FILE_PATH):
    load_dotenv(ENV_FILE_PATH)
    logging.info(".env encontrado y cargado.")
else:
    logging.error(f"Archivo .env NO encontrado en {ENV_FILE_PATH}.")
    raise FileNotFoundError(f"Asegúrate de que el archivo .env esté en la ruta correcta: {os.path.abspath(ENV_FILE_PATH)}")

GOOGLE_MAPS_API_KEYS = [os.getenv('GOOGLE_MAPS_API_KEY_1')] # Y las demás que tengas
GOOGLE_MAPS_API_KEYS = [key for key in GOOGLE_MAPS_API_KEYS if key]
if not GOOGLE_MAPS_API_KEYS:
    raise ValueError("No se encontraron claves API de Google Maps en el .env.")
logging.info(f"{len(GOOGLE_MAPS_API_KEYS)} clave(s) API de Google Maps cargada(s).")

2025-10-07 00:24:37,878 - INFO - Cargando variables de entorno desde: ../.env/.env
2025-10-07 00:24:37,880 - INFO - .env encontrado y cargado.
2025-10-07 00:24:37,880 - INFO - 1 clave(s) API de Google Maps cargada(s).


In [9]:
# --- Celda 4: Gestión de Clientes API y Rotación (SIMPLIFICADA) ---
logging.info("Configurando clientes de la API de Google Maps y lógica de rotación.")

if not GOOGLE_MAPS_API_KEYS:
    raise ValueError("La lista de claves API está vacía. No se pueden crear clientes.")

# Creamos una cola (deque) de clientes de Google Maps, uno por cada clave
gmaps_clients = deque([googlemaps.Client(key=key) for key in GOOGLE_MAPS_API_KEYS])
logging.info(f"Se crearon {len(gmaps_clients)} clientes de Google Maps.")

def get_current_gmaps_client():
    """Devuelve el cliente activo actual (el primero de la cola)."""
    return gmaps_clients[0]

def rotate_gmaps_client():
    """Rota el cliente al final de la cola. Si solo hay uno, no hace nada visible."""
    if len(gmaps_clients) > 1:
        logging.warning("Rotando cliente de API. La clave actual puede haber alcanzado su límite.")
        gmaps_clients.rotate(-1) # Mueve el primer elemento al final
        logging.info(f"Nuevo cliente de API activo.")
        time.sleep(2) # Pequeña pausa al rotar
    else:
        logging.warning("Intento de rotar, pero solo hay una clave API disponible.")
        # Podríamos añadir una pausa más larga aquí si la única clave falla repetidamente
        time.sleep(10) 
        
    return get_current_gmaps_client()

logging.info("Gestor de clientes API listo.")

2025-10-07 00:24:37,889 - INFO - Configurando clientes de la API de Google Maps y lógica de rotación.
2025-10-07 00:24:37,890 - INFO - API queries_quota: 60
2025-10-07 00:24:37,891 - INFO - Se crearon 1 clientes de Google Maps.
2025-10-07 00:24:37,891 - INFO - Gestor de clientes API listo.


In [10]:
# --- Celda 5: Funciones de Extracción para Google Places (CORREGIDA) ---
logging.info("Definiendo funciones para buscar y obtener detalles de lugares.")

MAX_RETRIES_PER_KEY = 3

# (La función text_search_paginated se mantiene igual, no necesita cambios)
def text_search_paginated(query, client):
    """
    Realiza una búsqueda de texto y maneja la paginación para obtener
    todos los Place IDs relevantes.
    """
    all_place_ids = []
    try:
        logging.info(f"Realizando búsqueda inicial para: '{query}'")
        response = client.places(query=query)
        all_place_ids.extend([place['place_id'] for place in response.get('results', [])])
        
        next_page_token = response.get('next_page_token')
        
        while next_page_token:
            logging.info("Paginación: Obteniendo siguiente página de resultados...")
            time.sleep(2) # Pausa obligatoria de Google
            response = client.places(page_token=next_page_token)
            all_place_ids.extend([place['place_id'] for place in response.get('results', [])])
            next_page_token = response.get('next_page_token')
        
        logging.info(f"Búsqueda para '{query}' completada. Se encontraron {len(all_place_ids)} lugares.")
        return all_place_ids

    except googlemaps.exceptions.ApiError as e:
        logging.error(f"Error de API en Text Search para '{query}': {e}")
        if e.status == 'OVER_QUERY_LIMIT':
            return "API_ERROR"
        return []
    except Exception as e:
        logging.error(f"Error inesperado en Text Search para '{query}': {e}")
        return []

def get_place_details(place_id, client):
    """
    Obtiene los detalles de un lugar usando su Place ID, solicitando los campos específicos.
    """
    # --- LISTA DE CAMPOS CORREGIDA ---
    fields_to_request = [
        'place_id',
        'name',
        'formatted_address',
        'geometry',
        'type',  # CORRECCIÓN: 'types' a 'type' (singular)
        'rating',
        'user_ratings_total',
        'reviews',
        'price_level',
        'opening_hours',
        'website',
        'international_phone_number',
        'wheelchair_accessible_entrance',
        'reservable',
        'business_status'
    ]
    
    try:
        details = client.place(
            place_id=place_id, 
            fields=fields_to_request, 
            language='es'
        )
        return details.get('result', {})
    except googlemaps.exceptions.ApiError as e:
        logging.error(f"Error de API obteniendo detalles para Place ID '{place_id}': {e}")
        return "API_ERROR"

# (La función process_query_and_get_ids se mantiene igual, no necesita cambios)
def process_query_and_get_ids(query):
    """
    Orquesta la búsqueda de una consulta completa, manejando la rotación de claves.
    """
    initial_client = get_current_gmaps_client()
    
    while True:
        client = get_current_gmaps_client()
        place_ids = text_search_paginated(query, client)
        
        if place_ids == "API_ERROR":
            rotate_gmaps_client()
            if get_current_gmaps_client() == initial_client:
                logging.error(f"Todas las claves API han fallado para la búsqueda '{query}'.")
                return []
            continue
        
        return place_ids

2025-10-07 00:24:37,902 - INFO - Definiendo funciones para buscar y obtener detalles de lugares.


In [11]:
# --- Celda 6: Ejecución de la Extracción por Descubrimiento (Versión Extendida) ---

# --- LISTA DE LA COMPRA (VERSIÓN HIPER-LOCALIZADA Y EXTENDIDA) ---
SEARCH_QUERIES = [
    # Gastronomía y Vida Nocturna
    "restaurantes de comida típica colombiana en Cali", "restaurantes gourmet en Cali",
    "restaurantes económicos en Cali", "fritanga en Cali", "comida de mar en Cali",
    "plazas de mercado en Cali", "cafés de especialidad en Cali", "panaderías en Cali",
    "heladerías en Cali", "bares de salsa en Cali", "discotecas en Cali",
    "escuelas de salsa para turistas en Cali", "bares con música en vivo en Cali",
    # Cultura y Puntos de Interés
    "museos en Cali, Colombia", "iglesias históricas en Cali", "monumentos y estatuas en Cali",
    "teatros en Cali", "galerías de arte en Cali", "centros culturales en Cali",
    "bibliotecas públicas en Cali", "sitios turísticos en el barrio San Antonio, Cali",
    "Cerro Cristo Rey, Cali", "Gato del Río, Cali",
    # Alojamiento
    "hoteles 5 estrellas en Cali", "hoteles boutique en Cali", "hoteles económicos en Cali",
    "hostales en Cali", "apartahoteles en Cali",
    # Recreación y Naturaleza
    "parques en Cali, Colombia", "parques ecológicos cerca de Cali", "Río Pance, Cali",
    "Zoológico de Cali", "Jardín Botánico de Cali", "miradores en Cali",
    "lugares para senderismo cerca de Cali", "piscinas públicas en Cali",
    # Compras y Servicios
    "centros comerciales en Cali", "tiendas de artesanías en Cali",
    "tiendas de souvenirs en Cali", "casas de cambio en Cali",
    # Bienestar
    "spas en Cali", "gimnasios en Cali", "centros de yoga en Cali"
]

def load_processed_ids_from_csv():
    """Función auxiliar para leer los IDs del CSV si el estado no existe."""
    try:
        if os.path.exists(RAW_API_CSV_PATH):
            df_existente = pd.read_csv(RAW_API_CSV_PATH)
            logging.info(f"Archivo CSV existente encontrado. Cargando {len(df_existente['place_id'].unique())} IDs ya procesados.")
            return set(df_existente['place_id'].dropna().unique())
    except Exception as e:
        logging.error(f"No se pudo leer el archivo CSV para cargar IDs previos: {e}")
    return set()

def load_state():
    """Carga el estado desde JSON. Si no existe, intenta cargar desde el CSV."""
    if os.path.exists(STATE_FILE_PATH):
        with open(STATE_FILE_PATH, 'r') as f:
            state = json.load(f)
            # Aseguramos que los IDs estén en un set para búsquedas rápidas
            state['processed_place_ids'] = set(state.get('processed_place_ids', []))
            return state
    # Si no hay archivo de estado, creamos uno a partir del CSV existente
    return {'processed_place_ids': load_processed_ids_from_csv()}

def save_state(state):
    """Guarda el estado a JSON, convirtiendo el set a lista para la serialización."""
    state_to_save = state.copy()
    state_to_save['processed_place_ids'] = list(state.get('processed_place_ids', []))
    with open(STATE_FILE_PATH, 'w') as f:
        json.dump(state_to_save, f, indent=4)

def run_extraction():
    logging.info("--- INICIANDO EXTRACCIÓN POR DESCUBRIMIENTO (VERSIÓN ROBUSTA) ---")
    state = load_state()
    processed_place_ids = state.get('processed_place_ids', set())
    
    file_exists = os.path.exists(RAW_API_CSV_PATH)
    
    try:
        # Iteramos SIEMPRE sobre todas las consultas
        for i, query in enumerate(SEARCH_QUERIES):
            logging.info(f"\nProcesando consulta ({i+1}/{len(SEARCH_QUERIES)}): '{query}'")
            
            place_ids_found = process_query_and_get_ids(query)
            new_places_found = 0
            
            for place_id in place_ids_found:
                # La lógica clave: nos saltamos lo que ya tenemos
                if place_id in processed_place_ids:
                    logging.debug(f"Place ID {place_id} ya procesado. Omitiendo.")
                    continue

                new_places_found += 1
                logging.info(f"Nuevo lugar encontrado! Obteniendo detalles para Place ID: {place_id}")
                details = get_place_details(place_id, get_current_gmaps_client())
                
                if details and details != "API_ERROR":
                    # ... (la lógica para extraer y aplanar 'place_data' es la misma)
                    reviews_text = [r.get('text', '') for r in details.get('reviews', []) if r.get('text')]
                    place_data = {
                        'place_id': details.get('place_id'),
                        'nombre_google': details.get('name'),
                        'direccion': details.get('formatted_address'),
                        'latitud': details.get('geometry', {}).get('location', {}).get('lat'),
                        'longitud': details.get('geometry', {}).get('location', {}).get('lng'),
                        'categorias_google': ', '.join(details.get('types', [])),
                        'calificacion': details.get('rating'),
                        'total_calificaciones': details.get('user_ratings_total'),
                        'comentarios': " | ".join(reviews_text),
                        'nivel_precios': details.get('price_level'),
                        'horarios': json.dumps(details.get('opening_hours', {}), ensure_ascii=False),
                        'telefono': details.get('international_phone_number'),
                        'sitio_web': details.get('website'),
                        'entrada_accesible': details.get('wheelchair_accessible_entrance'),
                        'es_reservable': details.get('reservable'),
                        'estado_negocio': details.get('business_status')
                    }
                    
                    df_to_append = pd.DataFrame([place_data])
                    df_to_append.to_csv(RAW_API_CSV_PATH, mode='a', header=not file_exists, index=False)
                    file_exists = True
                    
                    processed_place_ids.add(place_id)
                
                elif details == "API_ERROR":
                    logging.warning(f"Error de API persistente para {place_id}. Rotando clave.")
                    rotate_gmaps_client()
                
                time.sleep(0.5)
            
            logging.info(f"Consulta '{query}' completada. Se encontraron {new_places_found} lugares nuevos.")
            # Guardamos el estado después de cada consulta completa
            state['processed_place_ids'] = processed_place_ids
            save_state(state)

    except KeyboardInterrupt:
        logging.warning("Extracción interrumpida por el usuario.")
    finally:
        save_state(state)
        logging.info(f"Proceso finalizado. Estado guardado. Total de lugares únicos en la base de datos: {len(processed_place_ids)}")

# --- Ejecutar la función principal ---
run_extraction()

2025-10-07 00:24:37,916 - INFO - --- INICIANDO EXTRACCIÓN POR DESCUBRIMIENTO (VERSIÓN ROBUSTA) ---
2025-10-07 00:24:37,977 - INFO - Archivo CSV existente encontrado. Cargando 2042 IDs ya procesados.
2025-10-07 00:24:37,983 - INFO - 
Procesando consulta (1/43): 'restaurantes de comida típica colombiana en Cali'
2025-10-07 00:24:37,985 - INFO - Realizando búsqueda inicial para: 'restaurantes de comida típica colombiana en Cali'
2025-10-07 00:24:39,029 - INFO - Paginación: Obteniendo siguiente página de resultados...
2025-10-07 00:24:41,654 - INFO - Paginación: Obteniendo siguiente página de resultados...
2025-10-07 00:24:44,416 - INFO - Búsqueda para 'restaurantes de comida típica colombiana en Cali' completada. Se encontraron 60 lugares.
2025-10-07 00:24:44,417 - INFO - Nuevo lugar encontrado! Obteniendo detalles para Place ID: ChIJqcFyKjKnMI4ReRJpOBtm9YQ
2025-10-07 00:24:45,123 - INFO - Consulta 'restaurantes de comida típica colombiana en Cali' completada. Se encontraron 1 lugares nue

In [12]:
# --- Carga del Dataset ---
try:
    data_folder = '../datos_brutos/api/'
    file_name = '001_Local_cali_api.csv'
    file_path = os.path.join(data_folder, file_name)
    
    df_api = pd.read_csv(file_path)
    logging.info(f"Dataset '{file_name}' cargado exitosamente.")
except FileNotFoundError:
    logging.error(f"Error: El archivo no se encontró en la ruta: {file_path}")

2025-10-07 00:30:13,993 - INFO - Dataset '001_Local_cali_api.csv' cargado exitosamente.


In [13]:
df_api.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2237 entries, 0 to 2236
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   place_id              2237 non-null   object 
 1   nombre_google         2237 non-null   object 
 2   direccion             2237 non-null   object 
 3   latitud               2237 non-null   float64
 4   longitud              2237 non-null   float64
 5   categorias_google     2237 non-null   object 
 6   calificacion          2138 non-null   float64
 7   total_calificaciones  2138 non-null   float64
 8   comentarios           2054 non-null   object 
 9   nivel_precios         328 non-null    float64
 10  horarios              2237 non-null   object 
 11  telefono              1797 non-null   object 
 12  sitio_web             975 non-null    object 
 13  entrada_accesible     1041 non-null   object 
 14  es_reservable         372 non-null    object 
 15  estado_negocio       

In [14]:
df_api.head(365)

,place_id,nombre_google,direccion,latitud,longitud,categorias_google,calificacion,total_calificaciones,comentarios,nivel_precios,horarios,telefono,sitio_web,entrada_accesible,es_reservable,estado_negocio
0,ChIJ4ym1rp-mMI4RdMtQYqwmVGE,La Casona Valluna,"Cra. 38d #4C - 54, Nueva Granada, Cali, Valle ...",3.426279,-76.546359,"establishment, food, point_of_interest, restau...",4.4,3865.0,"Terrible experiencia, las marranitas llegaron ...",2.0,"{""open_now"": true, ""periods"": [{""close"": {""day...",+57 310 4472694,http://www.lacasonavalluna.com/,True,True,OPERATIONAL
1,ChIJpT3NWaqnMI4RJ0B8s1ESQ40,Restaurante La Abuela,"Cl. 52 #13A-24, Comuna 8, Cali, Valle del Cauc...",3.444960,-76.499559,"establishment, food, point_of_interest, restau...",4.3,343.0,Es un restaurante el cual he conocido casi la ...,2.0,"{""open_now"": true, ""periods"": [{""close"": {""day...",+57 602 4382765,https://www.paginasamarillas.com.co/empresas/r...,NaN,False,OPERATIONAL
2,ChIJJSW8MIOmMI4Rtvn2QSI02is,La Comitiva,"Cl. 4 #3432, El Sindicato, Cali, Valle del Cau...",3.434618,-76.544993,"establishment, food, point_of_interest, restau...",4.7,2281.0,"Restaurante románticamente ambientado, situado...",3.0,"{""open_now"": false, ""periods"": [{""close"": {""da...",+57 315 0713719,https://restaurantelacomitiva.com/,True,True,OPERATIONAL
3,ChIJ66yQrX2mMI4RjMrUlemZdy0,El Zaguán de San Antonio,"Cra. 12 #1-29, San Cayetano, Cali, Valle del C...",3.446042,-76.540740,"establishment, food, point_of_interest, restau...",4.2,2600.0,Mi completa admiración hacia este lugar ya que...,2.0,"{""open_now"": false, ""periods"": [{""close"": {""da...",+57 602 8938021,https://www.facebook.com/pages/El-Zaguan-De-Sa...,False,True,OPERATIONAL
4,ChIJe3iuqS2nMI4RvZ7C18ALXFo,Mi Tierra - Restaurante colombiano,"Cl. 2 #12A-05, COMUNA 3, Cali, Valle del Cauca...",3.445091,-76.540113,"establishment, food, point_of_interest, restau...",4.5,260.0,"La comida es buena, el lugar está muy adornado...",NaN,"{""open_now"": false, ""periods"": [{""close"": {""da...",+57 301 1781322,https://www.facebook.com/mitierratradicion2017/,NaN,True,OPERATIONAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,ChIJNeouYVOmMI4RnzJwmzjYjeI,Panadería Matecaña,"Cra. 12 #42-103, Comuna 8, Cali, Valle del Cau...",3.448389,-76.504160,"bakery, establishment, food, point_of_interest...",4.3,544.0,Unos productos con muy buen sabor. | Siempre m...,NaN,"{""open_now"": false, ""periods"": [{""close"": {""da...",+57 316 5133927,NaN,NaN,NaN,OPERATIONAL
361,ChIJfXc4nHCoMI4RoVYXHG1xv9o,Panadería y Pastelería Lizpan,"COMUNA 6, Cali, Valle del Cauca, Colombia",3.488983,-76.490751,"bakery, establishment, food, point_of_interest...",4.2,526.0,"Muy buena atención, todo delicioso 🤗 | Primero...",NaN,"{""open_now"": false, ""periods"": [{""close"": {""da...",NaN,NaN,NaN,NaN,OPERATIONAL
362,ChIJq7IKZDSmMI4RMp8-Tm___3U,Panadería Brisas de Salomia,"Cra 2 #46 - 23, Esmeralda, Cali, Valle del Cau...",3.467035,-76.505836,"bakery, establishment, food, point_of_interest...",4.4,187.0,Muy buen sitio excelente comida | Es una buena...,1.0,"{""open_now"": false, ""periods"": [{""close"": {""da...",+57 602 4494529,NaN,NaN,NaN,OPERATIONAL
363,ChIJ24jxmeemMI4RFOvYfozAd1E,Panaderia La Panameña,"Cra. 32 #322, San Carlos, Cali, Valle del Cauc...",3.424586,-76.511840,"bakery, establishment, food, point_of_interest...",4.3,743.0,Si vas de paso y buscas desayunar aquí en la P...,1.0,"{""open_now"": true, ""periods"": [{""open"": {""day""...",NaN,NaN,True,NaN,OPERATIONAL
